# ServiceX Example

In [ ]:
import requests
from minio import Minio
import tempfile
import pyarrow.parquet as pq



## Submit the Transform Request
We will create a REST request that specifies a DID along with a list of columns we want extracted.

We also tell ServiceX that we want the resulting columns to be stored as parquet files in the object store

In [ ]:
    servicex_endpoint = "http://localhost:5000/servicex"
    response = requests.post(servicex_endpoint+"/transformation", json={
        "did": "mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00",
        "columns": "Electrons.pt(), Electrons.eta(), Electrons.phi(), Electrons.e(), Muons.pt(), Muons.eta(), Muons.phi(), Muons.e()",
        "image": "sslhep/servicex_func_adl_xaod_transformr:v0.4",
        "result-destination": "object-store",
        "result-format": "parquet",
        "chunk-size": 7000,
        "workers": 1
    })

    print(response.json())
    request_id = response.json()["request_id"]
    status_endpoint = servicex_endpoint+"/transformation/{}/status".format(request_id)


## Wait for the Transform to Complete

In [ ]:
status = requests.get(status_endpoint).json()
print("We have processed {} files there are {} remaining".format(status['files-processed'], status['files-remaining']))

In [ ]:
minio_endpoint = "localhost:9000"
minio_client = Minio(minio_endpoint,
                 access_key='miniouser',
                 secret_key='leftfoot1',
                 secure=False)
objects = minio_client.list_objects(request_id)
sample_file = list([file.object_name for file in objects])[0]
print(sample_file)

In [ ]:
from IPython.display import display, HTML

with tempfile.TemporaryDirectory() as tmpdirname:
    minio_client.fget_object(request_id,
                             sample_file,
                             sample_file)
    pa_table = pq.read_table(sample_file)

    display(pa_table.to_pandas())
    